<a href="https://colab.research.google.com/github/GioLomia/Jigsaw_Speach_Toxicity_Classifier/blob/master/Jigsaw_Unintended_Bias_in_Toxicity_Classification.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [0]:
import numpy as np # linear algebra
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv)
import lightgbm as lgb
from sklearn.metrics import mean_squared_error
from sklearn.metrics import roc_auc_score
from sklearn.model_selection import StratifiedKFold
from google.colab import files
import pandas as pd

import keras
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.layers import Dense, Input, LSTM, Embedding, Dropout, Activation, Conv1D, GRU, CuDNNGRU, CuDNNLSTM, BatchNormalization
from keras.layers import Bidirectional, GlobalMaxPool1D, MaxPooling1D, Add, Flatten, Bidirectional
from keras.layers import GlobalAveragePooling1D, GlobalMaxPooling1D, concatenate, SpatialDropout1D
from keras.models import Model, load_model
from keras import initializers, regularizers, constraints, optimizers, layers, callbacks
from keras import backend as K
from keras.engine import InputSpec, Layer
from keras.optimizers import Adam

from keras.callbacks import ModelCheckpoint, TensorBoard, Callback, EarlyStopping
# Input data files are available in the "../input/" directory.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

import os
import warnings
warnings.filterwarnings('ignore')

Using TensorFlow backend.


In [0]:
from googleapiclient.discovery import build
import io, os
from googleapiclient.http import MediaIoBaseDownload
from google.colab import auth
auth.authenticate_user()
drive_service = build('drive', 'v3')
results = drive_service.files().list(
        q="name = 'kaggle.json'", fields="files(id)").execute()
kaggle_api_key = results.get('files', [])
filename = "/content/.kaggle/kaggle.json"
os.makedirs(os.path.dirname(filename), exist_ok=True)
request = drive_service.files().get_media(fileId=kaggle_api_key[0]['id'])
fh = io.FileIO(filename, 'wb')
downloader = MediaIoBaseDownload(fh, request)
done = False
while done is False:
    status, done = downloader.next_chunk()
    print("Download %d%%." % int(status.progress() * 100))
os.chmod(filename, 600)


For more information, please see:
  * https://github.com/tensorflow/community/blob/master/rfcs/20180907-contrib-sunset.md
  * https://github.com/tensorflow/addons
If you depend on functionality not listed there, please file an issue.

Download 100%.


In [0]:
!mkdir ~/.kaggle
!cp /content/.kaggle/kaggle.json ~/.kaggle/kaggle.json
!ls
!kaggle competitions list

adc.json  sample_data
ref                                                deadline             category            reward  teamCount  userHasEntered  
-------------------------------------------------  -------------------  ---------------  ---------  ---------  --------------  
digit-recognizer                                   2030-01-01 00:00:00  Getting Started  Knowledge       2571            True  
titanic                                            2030-01-01 00:00:00  Getting Started  Knowledge      10418           False  
house-prices-advanced-regression-techniques        2030-01-01 00:00:00  Getting Started  Knowledge       4326           False  
imagenet-object-localization-challenge             2029-12-31 07:00:00  Research         Knowledge         36           False  
competitive-data-science-predict-future-sales      2019-12-31 23:59:00  Playground           Kudos       2630           False  
two-sigma-financial-news                           2019-07-15 23:59:00  Featured  

In [0]:
!kaggle competitions files -c jigsaw-unintended-bias-in-toxicity-classification
!ls
!kaggle competitions download  -c jigsaw-unintended-bias-in-toxicity-classification -p /content/kaggle
!mkdir root
%cd kaggle
!ls

name                    size  creationDate         
---------------------  -----  -------------------  
sample_submission.csv    1MB  2019-03-28 21:16:21  
test.csv                29MB  2019-03-28 21:16:21  
train.csv              778MB  2019-03-28 21:16:18  
adc.json  sample_data
  0% 0.00/221k [00:00<?, ?B/s]
100% 221k/221k [00:00<00:00, 68.2MB/s]
 41% 5.00M/12.1M [00:00<00:00, 35.6MB/s]
100% 12.1M/12.1M [00:00<00:00, 59.1MB/s]
 95% 258M/273M [00:02<00:00, 152MB/s]
100% 273M/273M [00:02<00:00, 101MB/s]
/content/kaggle
sample_submission.csv.zip  test.csv.zip  train.csv.zip


In [0]:
!unzip /content/kaggle/train.csv.zip
!unzip /content/kaggle/test.csv.zip
!unzip /content/kaggle/sample_submission.csv.zip
!ls

Archive:  /content/kaggle/train.csv.zip
  inflating: train.csv               
Archive:  /content/kaggle/test.csv.zip
  inflating: test.csv                
Archive:  /content/kaggle/sample_submission.csv.zip
  inflating: sample_submission.csv   
sample_submission.csv	   test.csv	 train.csv
sample_submission.csv.zip  test.csv.zip  train.csv.zip


In [0]:
train=pd.read_csv("/content/kaggle/train.csv")
test=pd.read_csv("/content/kaggle/test.csv")
sample=pd.read_csv("/content/kaggle/sample_submission.csv")

In [0]:
train['comment_text'] = train['comment_text'].astype(str)
test['comment_text'] = test['comment_text'].astype(str)

In [0]:
full_text = list(train['comment_text'].values) + list(test['comment_text'].values)

In [0]:
%%time
tk = Tokenizer(lower = True, filters='', num_words=30000)
tk.fit_on_texts(full_text)

In [0]:
train_tokenized = tk.texts_to_sequences(train['comment_text'])
test_tokenized = tk.texts_to_sequences(test['comment_text'])

In [0]:
max_len = 550
X_train = pad_sequences(train_tokenized, maxlen = max_len)
X_test = pad_sequences(test_tokenized, maxlen = max_len)

In [0]:
# Load the Drive helper and mount
from google.colab import drive
drive.mount('/content/drive')

In [0]:
%cd /content/drive/My Drive

/content/drive/My Drive


In [0]:
embedding_path = "/content/drive/My Drive/Colab Notebooks/Kaggle/glove.6B.300d.txt"
# /kaggle/input/embeddings/glove.840B.300d/glove.840B.300d.txt

In [0]:
embed_size = 300
max_features = 30000

In [0]:
def get_coefs(word,*arr): return word, np.asarray(arr, dtype='float32')
embedding_index = dict(get_coefs(*o.strip().split(" ")) for o in open(embedding_path))

word_index = tk.word_index
nb_words = min(max_features, len(word_index))
embedding_matrix = np.zeros((nb_words + 1, embed_size))
for word, i in word_index.items():
    if i >= max_features: continue
    embedding_vector = embedding_index.get(word)
    if embedding_vector is not None: embedding_matrix[i] = embedding_vector

In [0]:
y = np.where(train['target'] >= 0.5, True, False) * 1

In [0]:
y = train['target']

In [0]:
def build_model(lr=0.0, lr_d=0.0, spatial_dr=0.0,  dense_units=128, dr=0.1):
    file_path = "best_model.hdf5"
    check_point = ModelCheckpoint(file_path, monitor = "val_loss", verbose = 1,
                                  save_best_only = True, mode = "min")
    early_stop = EarlyStopping(monitor = "val_loss", mode = "min", patience = 3)
    
    inp = Input(shape = (max_len,))
    
    x = Embedding(max_features +1, embed_size, weights = [embedding_matrix], trainable = False)(inp)
    x=  Bidirectional(LSTM(units=100,return_sequences=True,recurrent_dropout=0.24))(x)
    x=  Bidirectional(LSTM(units=50,recurrent_dropout=0.14,return_sequences=True))(x)
    x1 = SpatialDropout1D(spatial_dr)(x)
    # from benchmark kernel
    x = Conv1D(128, 2, activation='relu', padding='same')(x)
    x = MaxPooling1D(5, padding='same')(x)
    x = Conv1D(228, 3, activation='relu', padding='same')(x)
    x = MaxPooling1D(5, padding='same')(x)
    x = Conv1D(328, 4, activation='relu', padding='same')(x)
    x = MaxPooling1D(40, padding='same')(x)
    x = Flatten()(x)
    
    x = Dropout(dr)(Dense(dense_units, activation='relu') (x))
    x = Dense(1)(x)
    model = Model(inputs = inp, outputs = x)
    model.compile(loss = "mae", optimizer = Adam(lr = lr, decay = lr_d), metrics = ["accuracy"])
    history = model.fit(X_train, y, batch_size = 500, epochs = 1, validation_split=0.2, 
                        verbose = 1, callbacks = [check_point, early_stop])
    model = load_model(file_path)
    return model

In [0]:
model = build_model(lr = 1e-2, lr_d = 1e-10, spatial_dr = 0.1, dr=0.2)

In [0]:
from subprocess import check_output
pred = model.predict(X_test, batch_size = 1024, verbose = 1)
sub['prediction'] = pred
sub.to_csv('submission.csv', index=False)